In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
DATA_PATH = './data/housing.csv'

In [3]:
df_data = pd.read_csv(DATA_PATH)

In [4]:
from splitter import split_train_test
from pipeline import build_preprocess_pipeline

(x_train, y_train), (x_test, y_test) = split_train_test(df_data)

columns = list(x_train)
preprocess_pipeline = build_preprocess_pipeline(columns)

x_train_processed = preprocess_pipeline.fit_transform(x_train)
x_test_processed = preprocess_pipeline.transform(x_test)
print(x_train_processed.shape)

(16512, 16)


## Grid search

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# 3 * 4 + 2 * 3 = 18 parameter sets
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

forest_model = RandomForestRegressor(random_state=42)
# 5 fold cross validation, 18 * 5 = 90 training
grid_search = GridSearchCV(forest_model, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(x_train_processed, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'max_features': [2, 4, 6, 8],
  

In [6]:
grid_search.best_params_

{'max_features': 8, 'n_estimators': 30}

In [7]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=8, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=30,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [8]:
def build_result_df(search):
    df_result = pd.DataFrame(search.cv_results_)
    df_result = df_result[['params', 'mean_test_score']]
    df_result['rmse'] = (-df_result['mean_test_score']) ** 0.5
    return df_result

In [9]:
df_result = build_result_df(grid_search)
df_result

,params,mean_test_score,rmse
0,"{'max_features': 2, 'n_estimators': 3}",-4.053749e+09,63669.057917
1,"{'max_features': 2, 'n_estimators': 10}",-3.094381e+09,55627.161713
2,"{'max_features': 2, 'n_estimators': 30}",-2.849913e+09,53384.578676
3,"{'max_features': 4, 'n_estimators': 3}",-3.716852e+09,60965.991859
4,"{'max_features': 4, 'n_estimators': 10}",-2.781611e+09,52740.982485
5,"{'max_features': 4, 'n_estimators': 30}",-2.537877e+09,50377.344410
6,"{'max_features': 6, 'n_estimators': 3}",-3.441447e+09,58663.847334
7,"{'max_features': 6, 'n_estimators': 10}",-2.704640e+09,52006.153560
8,"{'max_features': 6, 'n_estimators': 30}",-2.514668e+09,50146.465964
9,"{'max_features': 8, 'n_estimators': 3}",-3.348851e+09,57869.255040


## Random search

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
    'n_estimators': randint(low=1, high=200),
    'max_features': randint(low=1, high=8),
}

forest_model = RandomForestRegressor(random_state=42)
random_search = RandomizedSearchCV(
    forest_model, param_distributions=param_distribs, 
    n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(x_train_processed, y_train)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...


In [11]:
df_random_search_result = build_result_df(random_search)
df_random_search_result

,params,mean_test_score,rmse
0,"{'max_features': 7, 'n_estimators': 180}",-2.415787e+09,49150.657233
1,"{'max_features': 5, 'n_estimators': 15}",-2.640917e+09,51389.852957
2,"{'max_features': 3, 'n_estimators': 72}",-2.580246e+09,50796.120460
3,"{'max_features': 5, 'n_estimators': 21}",-2.584207e+09,50835.099320
4,"{'max_features': 7, 'n_estimators': 122}",-2.428607e+09,49280.901179
5,"{'max_features': 3, 'n_estimators': 75}",-2.578087e+09,50774.866790
6,"{'max_features': 3, 'n_estimators': 88}",-2.568741e+09,50682.750012
7,"{'max_features': 5, 'n_estimators': 100}",-2.461047e+09,49608.940613
8,"{'max_features': 3, 'n_estimators': 150}",-2.547582e+09,50473.576428
9,"{'max_features': 5, 'n_estimators': 2}",-4.151194e+09,64429.763805


## Feature importance

In [12]:
# Numerical features
num_features = list(df_data.columns)
num_features.remove('ocean_proximity')
num_features.remove('median_house_value')
# Extra features added through preprocess
extra_features = ['rooms_per_hhold', 'pop_per_hhold', 'bedrooms_per_room']
# Categorical features
ocean_proximity_categories = list(preprocess_pipeline.named_transformers_['categorical'].categories_[0])

features = num_features + extra_features + ocean_proximity_categories
importance = grid_search.best_estimator_.feature_importances_

df_feature_importance = pd.DataFrame({'features': features, 'importance': importance})
df_feature_importance.sort_values(by=['importance'], ascending=False, inplace=True)
df_feature_importance.reset_index(drop=True, inplace=True)
df_feature_importance

,features,importance
0,median_income,0.366159
1,INLAND,0.164781
2,pop_per_hhold,0.108793
3,longitude,0.073344
4,latitude,0.062909
5,rooms_per_hhold,0.056419
6,bedrooms_per_room,0.053351
7,housing_median_age,0.041144
8,population,0.014874
9,total_rooms,0.014673


## Final model

In [13]:
from sklearn.metrics import mean_squared_error

final_model = grid_search.best_estimator_

final_predictions = final_model.predict(x_test_processed)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print(final_rmse)

47730.22690385927


### Confidence interval (T-score)

#### Use scipy

In [14]:
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
mean = squared_errors.mean()
m = len(squared_errors)

np.sqrt(stats.t.interval(confidence, m - 1, loc=np.mean(squared_errors), scale=stats.sem(squared_errors)))

array([45685.10470776, 49691.25001878])

#### Manual calculation

In [15]:
tscore = stats.t.ppf((1 + confidence) / 2, df=m - 1)
tmargin = tscore * squared_errors.std(ddof=1) / np.sqrt(m)
np.sqrt(mean - tmargin), np.sqrt(mean + tmargin)

(45685.10470776014, 49691.25001877871)

### Confidence interval (Z-score)

In [16]:
zscore = stats.norm.ppf((1 + confidence) / 2)
zmargin = zscore * squared_errors.std(ddof=1) / np.sqrt(m)
np.sqrt(mean - zmargin), np.sqrt(mean + zmargin)

(45685.717918136594, 49690.68623889426)